## Setting up the development environment

In [1]:
!pip install cohere tiktoken openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00


In [2]:
!pip install python-dotenv

In [3]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_key  = os.environ["OPENAI_API_KEY"]

## Loading data

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

file_name = "california_tour_package.csv"

df = pd.read_csv(file_name)
df.shape

(32, 3)

In [5]:
df.head()

,Tour Name,Tour Blurb,Tour Description
0,Big Sur Retreat,Big Sur is big country. The Big Sur Retreat takes you to the most majestic part of the Pacific Coast and show you the secret trails.,"The region know as Big Sur is like Yosemite's younger cousin, with all the redwood scaling, rock climbing and, best of all, hiking that the larger park has to offer. Robison Jeffers once said, ""Big Sur is the greatest meeting of land and sea in the world,"" but the highlights are only accessible on foot.\nOur 3-day tour allows you to choose from multiple hikes led by experienced guides during the day, while comfortably situated in the evenings at the historic Big Sur River Inn. Take a tranquil walk to the coastal waterfall at Julia Pfeiffer Burns State Par or hike to the Married Redwoods. If you're prepared for a more strenuous climb, try Ollason's Peak in Toro Park. An optional 4th day includes admission to the Henry Miller Library and the Point Reyes Lighthouse."
1,In the Steps of John Muir,"Follow in the steps on John Muir, famous naturalist and founder of the Sierra Club, and walk the same trails he helped blaze in and around Yosemite National Park.","John Muir famously walked from San Francisco to Yosemite, an astounding 300 miles, and this tour is designed to trace his steps-- at least part of the way.\nThis tour is not for the faint of heart though. This is a true backpacker's adventure, 37 miles in 3 days. Along the way, you'll bear witness to the classic monuments of Yosemite, such as Vernal Falls, Nevada Falls, Half Dome, Cathedral Peak, Tuolumne Meadows and Mt. Lyell. At Nevada Falls, the trail becomes narrow and from then on, you'll be privy to a ""secret"" Yosemite, including a rarely seen face of Half Dome. John Muir Trail tickets are required- and hard to come by- but Explore California has you covered. We'll also provide a checklist so that you come prepared with the right equipment and a seasoned guide for your tour."
2,The Death Valley Survivor's Trek,Hot stuff? Need more of a challenge? Take this tour to the hottest place in North America: Death Valley.,"This 2 day, 1 night tour takes you deep into the heart of Death Valley National Park. Due to extreme temperatures (120 degrees and higher) in the summer months, this tour is only offered November through April. \nThis overnighter is no picnic though. Our guide will lead you on a 12 mile, off-trail hike down Death Valley Wash and into the backcountry for an overnight, hard terrain camping. Luckily, 12 miles takes you out of range of almost any city lights, something that draws thousands of star gazers to Death Valley ever year. If you can survive the 12 miles back, take advantage of the location to stop in at Scotty's Castle, a genuine castle built by the eccentric and wealthy Walter Scott for his wife. This tour will make you truly appreciate modern luxuries."
3,The Mt. Whitney Climbers Tour,"Climb to the sky! The Mt. Whitney Climbers Tour takes you to the top of this 14,000 ft. of mountain in 4 days- our longest and most strenuous backpacking tour.","Backpackers and hikers taking the Mt. Whitney Climbers Tour won't need hooks and rappelling gear, but they will need to be on the look out for majestic sunrises, giant granite peaks and everything from brown bears to marmots. You'll approach the towering peak from the trailhead on the western face, starting 6,000 feet above sea level. The Mt. Whitney Trail extends 11 miles and another 8 feet up to the peak. While no expensive training is required, you'll still need planning, permits, and preparation. Explore California will set you up with a trail permit, a two night camping pass, and an expert guide. We'll also help outfit you with the correct gear and make sure you are prepared for the length and elevation gain of the climb."
4,Channel Islands Excursion,"The chain known as the Channel Islands offer some of the most diverse and unique landscape on the Pacific coast. No motor vehicles are allowed on the islands, which make

In [6]:
df.iloc[[1]]

,Tour Name,Tour Blurb,Tour Description
1,In the Steps of John Muir,"Follow in the steps on John Muir, famous naturalist and founder of the Sierra Club, and walk the same trails he helped blaze in and around Yosemite National Park.","John Muir famously walked from San Francisco to Yosemite, an astounding 300 miles, and this tour is designed to trace his steps-- at least part of the way.\nThis tour is not for the faint of heart though. This is a true backpacker's adventure, 37 miles in 3 days. Along the way, you'll bear witness to the classic monuments of Yosemite, such as Vernal Falls, Nevada Falls, Half Dome, Cathedral Peak, Tuolumne Meadows and Mt. Lyell. At Nevada Falls, the trail becomes narrow and from then on, you'll be privy to a ""secret"" Yosemite, including a rarely seen face of Half Dome. John Muir Trail tickets are required- and hard to come by- but Explore California has you covered. We'll also provide a checklist so that you come prepared with the right equipment and a seasoned guide for your tour."


In [7]:
content = df.iloc[[1]]['Tour Description']
print(content)

1    John Muir famously walked from San Francisco to Yosemite, an astounding 300 miles, and this tour is designed to trace his steps-- at least part of the way.\nThis tour is not for the faint of heart though. This is a true backpacker's adventure, 37 miles in 3 days. Along the way, you'll bear witness to the classic monuments of Yosemite, such as Vernal Falls, Nevada Falls, Half Dome, Cathedral Peak, Tuolumne Meadows and Mt. Lyell. At Nevada Falls, the trail becomes narrow and from then on, you'll be privy to a "secret" Yosemite, including a rarely seen face of Half Dome. John Muir Trail tickets are required- and hard to come by- but Explore California has you covered. We'll also provide a checklist so that you come prepared with the right equipment and a seasoned guide for your tour.
Name: Tour Description, dtype: object


## Entity detection

In [8]:
prompt_template = """
Identify the following items from the tour package description:
- Attractions
- Days of the tour
- Hotel

The tour package description is delimited with triple backticks.
If the information isn't mentioned, return "NaN" as the value.
Be concise.

Tour Description: ```{content}```
"""

In [9]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=llm_name, temperature=1)

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

In [10]:
results = llm_chain(content)

In [11]:
print(results['text'])

Attractions: Vernal Falls, Nevada Falls, Half Dome, Cathedral Peak, Tuolumne Meadows, Mt. Lyell, "secret" face of Half Dome
Days of the tour: 3
Hotel: NaN


In [12]:
ans = llm_chain.predict(content=content)
print(ans)

Attractions: Vernal Falls, Nevada Falls, Half Dome, Cathedral Peak, Tuolumne Meadows, Mt. Lyell. 

Days of the tour: 3 days. 

Hotel: NaN.


In [13]:
df.iloc[[12]]

,Tour Name,Tour Blurb,Tour Description
12,Paso Robles Artesian Spring,"Paso Robles is traditionally know as wine country, but there's something else this region has to offer: incredible thermal hot springs. This package is our most plush hot springs tour, where you can while away the day in the luxurious surroundings at the River Oaks Spa and choose from one of their many specialized spa treatments.","Situated in the midst of rolling hills and vineyards, Paso Robles has been renowned for thermal springs water. The Salinan Indians, the most historical inhabitants of the area, knew this area as “The Springs” and found relief from various ailments in the therapeutic waters, which arise naturally at 117 degrees. At River Oaks Hot Springs Spa, you have access to these springs while surrounded by a world-class spa facility. Choose from a private indoor, open-air tub or an outdoor hillside tub, and select from among their massage or face and skin therapy services. Spend the night at the Orchard Hill Bed & Breakfast and spend the following day wandering the wineries or antiquing in the quaint downtown."


In [14]:
content = df.iloc[[12]]['Tour Description']

ans = llm_chain.predict(content=content)
print(ans)

Attractions: River Oaks Hot Springs Spa, Orchard Hill Bed & Breakfast, wineries, quaint downtown
Days of the tour: 2 (1 night at Orchard Hill Bed & Breakfast)
Hotel: Orchard Hill Bed & Breakfast


In [15]:
df.iloc[[25]]

,Tour Name,Tour Blurb,Tour Description
25,Mammoth Mountain Adventure,"With over 400 inches of annual snowfall, Mammoth Mountain turns winter into a 3 season event! Learn to ski, snowboard or even snowshoe across the thousands of acres of this winter wonderland.","The Mammoth Mountain Adventure is 4 days of wild, winter fun. Take a lesson the first day from the Ski & Snowboard School, led by grizzly pro instructors. This tour includes a 4 full day lift pass, so when you're ready, you can take on any one of the 150 trails over Mammoth's epic terrain. Equipment rentals are included, so you can pack light. Need a break? Take a scenic gondola to the summit of Mammoth Mountain, 11,000 feet in all, or take a Snowcat tour of the backcountry. Enjoy dinners at the Mammoth Mountain Inn, a snowball's throw from the main lodge, where you'll also be provided 3 night's complimentary lodging. This package is available October through early June, but lift tickets are rare, so book now."


In [16]:
content = df.iloc[[25]]['Tour Description']

ans = llm_chain.predict(content=content)
print(ans)

Attractions: Ski & Snowboard School, Mammoth Mountain trails, scenic gondola, Snowcat tour
Days of the tour: 4
Hotel: Mammoth Mountain Inn
